## Train a SVC classifier by your own pics.
用自己家人的照片訓練一個classifier

In [1]:
"""Functions for building the face recognition network.

   facenet.py
   
"""
# MIT License
# 
# Copyright (c) 2016 David Sandberg
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# pylint: disable=missing-docstring
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from subprocess import Popen, PIPE
import tensorflow as tf
import numpy as np
from scipy import misc
import imageio                        # scipy.misc.imsave has been deprecated in newer Scipy versions.
                                      # try this if you got error on 'scipy.misc' has no attribute 'imread'
from sklearn.model_selection import KFold
from scipy import interpolate
from tensorflow.python.training import training
import random
import re
from tensorflow.python.platform import gfile
import math
from six import iteritems


def get_dataset(path, has_class_directories=True):
    dataset = []
    path_exp = os.path.expanduser(path)
    classes = [path for path in os.listdir(path_exp) \
                    if os.path.isdir(os.path.join(path_exp, path))]
    classes.sort()
    nrof_classes = len(classes)
    for i in range(nrof_classes):
        class_name = classes[i]
        facedir = os.path.join(path_exp, class_name)
        image_paths = get_image_paths(facedir)
        dataset.append(ImageClass(class_name, image_paths))
  
    return dataset

def get_image_paths(facedir):
    image_paths = []
    if os.path.isdir(facedir):
        images = os.listdir(facedir)
        image_paths = [os.path.join(facedir,img) for img in images]
    return image_paths

def get_image_paths_and_labels(dataset):
    image_paths_flat = []
    labels_flat = []
    for i in range(len(dataset)):
        image_paths_flat += dataset[i].image_paths
        labels_flat += [i] * len(dataset[i].image_paths)
    return image_paths_flat, labels_flat

def load_model(model, input_map=None):
    # Check if the model is a model directory (containing a metagraph and a checkpoint file)
    #  or if it is a protobuf file with a frozen graph
    src_path = os.path.join(os.getcwd(), model) # download pre-trained facenet ResNet model, put under the "models" folder
    model_exp = os.path.expanduser(src_path)
    if (os.path.isfile(model_exp)):
        print('Model filename: %s' % model_exp)
        with gfile.FastGFile(model_exp,'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            tf.import_graph_def(graph_def, input_map=input_map, name='')
    else:
        print('Model directory: %s' % model_exp)
        meta_file, ckpt_file = get_model_filenames(model_exp)
        
        print('Metagraph file: %s' % meta_file)
        print('Checkpoint file: %s' % ckpt_file)
      
        saver = tf.train.import_meta_graph(os.path.join(model_exp, meta_file), input_map=input_map)
        saver.restore(tf.get_default_session(), os.path.join(model_exp, ckpt_file))
        
def prewhiten(x):
    mean = np.mean(x)
    std = np.std(x)
    std_adj = np.maximum(std, 1.0/np.sqrt(x.size))
    y = np.multiply(np.subtract(x, mean), 1/std_adj)
    return y  
        
def load_data(image_paths, do_random_crop, do_random_flip, image_size, do_prewhiten=True):
    nrof_samples = len(image_paths)
    images = np.zeros((nrof_samples, image_size, image_size, 3))
    for i in range(nrof_samples):
        #img = misc.imread(image_paths[i])
        img = imageio.imread(image_paths[i])         # try this if you got error on 'scipy.misc' has no attribute 'imread'
        if img.ndim == 2:
            img = to_rgb(img)
        if do_prewhiten:
            img = prewhiten(img)
        img = crop(img, do_random_crop, image_size)
        img = flip(img, do_random_flip)
        images[i,:,:,:] = img
    return images

def crop(image, random_crop, image_size):
    if image.shape[1]>image_size:
        sz1 = int(image.shape[1]//2)
        sz2 = int(image_size//2)
        if random_crop:
            diff = sz1-sz2
            (h, v) = (np.random.randint(-diff, diff+1), np.random.randint(-diff, diff+1))
        else:
            (h, v) = (0,0)
        image = image[(sz1-sz2+v):(sz1+sz2+v),(sz1-sz2+h):(sz1+sz2+h),:]
    return image

def flip(image, random_flip):
    if random_flip and np.random.choice([True, False]):
        image = np.fliplr(image)
    return image

class ImageClass():
    "Stores the paths to images for a given class"
    def __init__(self, name, image_paths):
        self.name = name
        self.image_paths = image_paths
  
    def __str__(self):
        return self.name + ', ' + str(len(self.image_paths)) + ' images'
  
    def __len__(self):
        return len(self.image_paths)
    
def get_model_filenames(model_dir):
    files = os.listdir(model_dir)
    meta_files = [s for s in files if s.endswith('.meta')]
    if len(meta_files)==0:
        raise ValueError('No meta file found in the model directory (%s)' % model_dir)
    elif len(meta_files)>1:
        raise ValueError('There should not be more than one meta file in the model directory (%s)' % model_dir)
    meta_file = meta_files[0]
    ckpt = tf.train.get_checkpoint_state(model_dir)
    if ckpt and ckpt.model_checkpoint_path:
        ckpt_file = os.path.basename(ckpt.model_checkpoint_path)
        return meta_file, ckpt_file

    meta_files = [s for s in files if '.ckpt' in s]
    max_step = -1
    for f in files:
        step_str = re.match(r'(^model-[\w\- ]+.ckpt-(\d+))', f)
        if step_str is not None and len(step_str.groups())>=2:
            step = int(step_str.groups()[1])
            if step > max_step:
                max_step = step
                ckpt_file = step_str.groups()[0]
    return meta_file, ckpt_file

C:\Users\user\anaconda3\envs\py37_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\anaconda3\envs\py37_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\anaconda3\envs\py37_tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\anaconda3\envs\py37_tf\lib\site-packages\tensorflow\pyth

In [2]:
import argparse

parser = argparse.ArgumentParser()

parser.add_argument('--mode', type=str, #choices=['TRAIN', 'CLASSIFY'],
    help='Indicates if a new classifier should be trained or a classification ' + 
    'model should be used for classification', default='TRAIN')  # default could be either 'TRAIN' or 'CLASSIFY' , choices=['TRAIN', 'CLASSIFY']
parser.add_argument('--data_dir', type=str, default=r"input_dir",
    help='Path to the data directory containing aligned LFW face patches.')
parser.add_argument('--model', type=str, default=r"models",
    help='Could be either a directory containing the meta_file and ckpt_file or a model protobuf (.pb) file')
parser.add_argument('--classifier_filename', default="my_classifier.pkl",
    help='Classifier model file name as a pickle (.pkl) file. ' + 
    'For training this is the output and for classification this is an input.')
parser.add_argument('--use_split_dataset', 
    help='Indicates that the dataset specified by data_dir should be split into a training and test set. ' +  
    'Otherwise a separate test set can be specified using the test_data_dir option.', action='store_true')
parser.add_argument('--test_data_dir', type=str,  default=r"input_dir",
    help='Path to the test data directory containing aligned images used for testing.')
parser.add_argument('--batch_size', type=int,
    help='Number of images to process in a batch.', default=90)
parser.add_argument('--image_size', type=int,
    help='Image size (height, width) in pixels.', default=160)
parser.add_argument('--seed', type=int,
    help='Random seed.', default=666)
parser.add_argument('--min_nrof_images_per_class', type=int,
    help='Only include classes with at least this number of images in the dataset', default=25)
parser.add_argument('--nrof_train_images_per_class', type=int,
    help='Use this number of images from each class for training and the rest for testing', default=15)

args = parser.parse_args(args=["--mode", "TRAIN", "--data_dir", "input_dir", "--model", "models", "--classifier_filename", "my_classifier.pkl",
                               "--use_split_dataset", "--test_data_dir", r"input_dir", "--batch_size", "90", "--image_size", "160",
                               "--seed", "666", "--min_nrof_images_per_class", "25", "--nrof_train_images_per_class", "15"])


In [3]:
"""An example of how to use your own dataset to train a classifier that recognizes people.

   classifier.py
"""
# MIT License
# 
# Copyright (c) 2016 David Sandberg
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
# 
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
# 
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

#import facenet
import sys
import pickle
from sklearn.svm import SVC

def main(args):
  
    with tf.Graph().as_default():
      
        with tf.Session() as sess:
            
            np.random.seed(seed=args.seed)
            
            if args.use_split_dataset:
                #dataset_tmp = facenet.get_dataset(args.data_dir)
                dataset_tmp = get_dataset(args.data_dir)
                train_set, test_set = split_dataset(dataset_tmp, args.min_nrof_images_per_class, args.nrof_train_images_per_class)
                if (args.mode=='TRAIN'):
                    dataset = train_set         # train mode, load train_set
                elif (args.mode=='CLASSIFY'):
                    dataset = test_set          # classify mode, load test_set
            else:
                #dataset = facenet.get_dataset(args.data_dir)
                dataset = get_dataset(args.data_dir)

            # Check that there are at least one training image per class
            for cls in dataset:
                assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')            

                 
            #paths, labels = facenet.get_image_paths_and_labels(dataset)
            paths, labels = get_image_paths_and_labels(dataset)
            
            print('Number of classes: %d' % len(dataset))
            print('Number of images: %d' % len(paths))
            
            # Load the model
            print('Loading feature extraction model')
            #facenet.load_model(args.model)
            load_model(args.model)             # load facenet Inception ResNet v1 model
            
            # Get input and output tensors
            images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
            embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
            embedding_size = embeddings.get_shape()[1]
            
            # Run forward pass to calculate embeddings
            print('Calculating features for images')
            nrof_images = len(paths)
            nrof_batches_per_epoch = int(math.ceil(1.0*nrof_images / args.batch_size))
            emb_array = np.zeros((nrof_images, embedding_size))
            for i in range(nrof_batches_per_epoch):
                start_index = i*args.batch_size
                end_index = min((i+1)*args.batch_size, nrof_images)
                paths_batch = paths[start_index:end_index]
                #images = facenet.load_data(paths_batch, False, False, args.image_size)
                images = load_data(paths_batch, False, False, args.image_size)
                feed_dict = { images_placeholder:images, phase_train_placeholder:False }
                emb_array[start_index:end_index,:] = sess.run(embeddings, feed_dict=feed_dict)
            
            src_path = os.path.join(os.getcwd(), args.model)
            classifier_filename_exp = os.path.join(src_path, args.classifier_filename) # define .pkl file path

            if (args.mode=='TRAIN'):                                # train mode
                # Train classifier
                print('Training classifier')
                model = SVC(kernel='linear', probability=True)     # train classifier with SVC
                model.fit(emb_array, labels)                        # labels assigned by dataset folders priority, 0, 1, 2 ...
            
                # Create a list of class names
                class_names = [cls.name.replace('_', ' ') for cls in dataset]

                # Saving classifier model
                with open(classifier_filename_exp, 'wb') as outfile:
                    pickle.dump((model, class_names), outfile)      # output SVC weights as a pkl file
                print('Saved classifier model to file "%s"' % classifier_filename_exp)
                
            elif (args.mode=='CLASSIFY'):                          # classify mode
                # Classify images
                print('Testing classifier')
                with open(classifier_filename_exp, 'rb') as infile:
                    (model, class_names) = pickle.load(infile)     # load SVC model weight

                print('Loaded classifier model from file "%s"' % classifier_filename_exp)

                predictions = model.predict_proba(emb_array)         # predict by embedding vector (softmax result)
                best_class_indices = np.argmax(predictions, axis=1)  # get the index of best score from softmax result
                best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices] # [item_num, best_class_indices]
                
                for i in range(len(best_class_indices)):
                    print('%4d  %s: %.3f' % (i, class_names[best_class_indices[i]], best_class_probabilities[i]))
                    
                accuracy = np.mean(np.equal(best_class_indices, labels))
                print('Accuracy: %.3f' % accuracy)
                
            
def split_dataset(dataset, min_nrof_images_per_class, nrof_train_images_per_class):
    train_set = []
    test_set = []
    for cls in dataset:
        paths = cls.image_paths
        # Remove classes with less than min_nrof_images_per_class
        if len(paths)>=min_nrof_images_per_class:
            np.random.shuffle(paths)
            #train_set.append(facenet.ImageClass(cls.name, paths[:nrof_train_images_per_class]))
            #test_set.append(facenet.ImageClass(cls.name, paths[nrof_train_images_per_class:]))
            train_set.append(ImageClass(cls.name, paths[:nrof_train_images_per_class]))
            test_set.append(ImageClass(cls.name, paths[nrof_train_images_per_class:]))
    return train_set, test_set

<>:54: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:54: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:54: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-3-b587b2e85074>:54: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')


### Create dataset input directory before you run
<br>1. Need to creat a "input_dir" folder as a Input path 建立 "input_dir" 資料夾, 把要輸入的照片(160x160)放裡面
<br>2. Need to creat a "model" folder, put the download model & weight(<a href="https://drive.google.com/file/d/1EXPBSXwTaqrSC0OhUdXNmKSh9qJUQ55-/view">20180402-114759</a>) here, 建立 "model" 資料夾, 把下載的模型參數放這
<br>We will get a classifier model(.pkl), will also output in the model directory, 等下產出的classifier模型參數(.pkl)也會在這出現
<br><p style="text-align:left;"><img src="images/dir_struc.png"  style="width:861px;height:496px;" align="left">
<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
<br>3-1. Train or Classify, when you want to train a new classifier, define the args.modle = TRAIN, in the above cell
<br>   args = parser.parse_args(args=["--mode", <b style="font-size:100%;">"TRAIN", </b>...
<br>   訓練時須將上方code改成 "--mode"部分的default值設定為 TRAIN
<br>3-2. Train or Classify, when you want to test the classifier, define the args.modle = CLASSIFY, in the above cell
<br>   args = parser.parse_args(args=["--mode", <b style="font-size:100%;">"CLASSIFY", </b>...
<br>   訓練完成後須將上方code改成 "--mode"部分的default值設定為 CLASSIFY, 進行TEST

<br>4-1. set your default value for min_nrof_images_per_class, in the above cell
<br>   args = parser.parse_args(args=[,<b style="font-size:100%;">"--min_nrof_images_per_class", "25"</b>,...
<br>   input_dir中, 相片少於這個數目的類別, 不會輸入
<br>4-2. set your default value for nrof_train_images_per_class, in the above cell
<br>   args = parser.parse_args(args=[,<b style="font-size:100%;">"--nrof_train_images_per_class", "15"</b>,...
<br>   input_dir中, 每個輸入SVC classifier的類別, 會用其中一部分作為training set使用, 剩餘的作為test set使用

In [4]:
main(args) # initiate main function

Number of classes: 4
Number of images: 60
Loading feature extraction model
Model directory: C:\Users\user\face_recog\DGEs\facenet_upload\step3_classifier\models
Metagraph file: model-20180402-114759.meta
Checkpoint file: model-20180402-114759.ckpt-275
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\user\face_recog\DGEs\facenet_upload\step3_classifier\models\model-20180402-114759.ckpt-275
Calculating features for images
Training classifier
Saved classifier model to file "C:\Users\user\face_recog\DGEs\facenet_upload\step3_classifier\models\my_classifier.pkl"
